In [ ]:
!pip install geopy

In [1]:
import doctest
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.datasets import make_blobs
from scipy.stats import multivariate_normal
from matplotlib.patches import Ellipse
from sklearn.neighbors import NearestNeighbors
from geopy.distance import geodesic

# two useful data viz libraries
import matplotlib.pyplot as plt
import seaborn as sns
# setup plotting in a notebook in a reasonable way
%matplotlib inline
%config InlineBackend.figure_format='retina'

# default figure aesthetics I'll be using, 
# there are other choices, see seaborn docs
sns.set_style("white")
sns.set_context("notebook")

In [2]:
cleaned_housing_data = pd.read_csv('cleaned_housing_data.csv')
cleaned_housing_data

,State,City,Street,Zipcode,Bedroom,Bathroom,Area,PPSq,LotArea,MarketEstimate,...,ListedPrice,population,density,source,military,incorporated,timezone,ranking,zips,id
0,AL,Saraland,Scott Dr,36571.0,4.0,2.0,1614.0,148.636927,0.3805,240600.0,...,239900.0,16109,193.5,shape,False,True,America/Chicago,3,36571,1840015886
1,AL,Saraland,Deer Run Dr,36571.0,3.0,2.0,1828.0,108.315098,0.4658,199700.0,...,198000.0,16109,193.5,shape,False,True,America/Chicago,3,36571,1840015886
2,AL,Chelsea,Mallet Way,35043.0,3.0,3.0,2224.0,150.629496,0.2600,336200.0,...,335000.0,15201,181.7,shape,False,True,America/Chicago,3,35051 35043,1840014823
3,AL,Chelsea,Highway 440,35043.0,3.0,2.0,1425.0,195.789474,1.8200,280200.0,...,279000.0,15201,181.7,shape,False,True,America/Chicago,3,35051 35043,1840014823
4,AL,Montgomery,Brampton Ln,36117.0,3.0,2.0,1564.0,96.547315,0.2000,150500.0,...,151000.0,251158,482.6,shape,False,True,America/Chicago,2,36112 36113 36110 36111 36116 36114 36115 3610...,1840008353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12599,WY,Big Piney,Taylor Ave,83113.0,4.0,2.0,1820.0,112.637363,0.1600,189800.0,...,205000.0,450,342.1,shape,False,True,America/Denver,3,83113,1840020061
12600,WY,Daniel,Booth Dr,83115.0,1.0,1.0,1008.0,337.202381,10.0600,307500.0,...,339900.0,158,12.0,shape,False,False,America/Denver,3,83115,1840018659
12601,WY,Shoshoni,Pine St,82649.0,6.0,3.0,3294.0,74.377656,0.3200,236800.0,...,245000.0,640,69.0,shape,False,True,America/Denver,3,82649,1840022355
12602,WY,Mills,N 4th Ave,82604.0,4.0,2.0,2120.0,90.094340,0.3100,184600.0,...,191000.0,4218,319.8,shape,False,True,America/Denver,3,82644 82604 82601,1840022359


In [3]:
major_city_df = pd.read_csv('major_city_df.csv')
major_city_df

,city,state_id,population,lat,lng
0,New York,NY,18908608,40.6943,-73.9249
1,Los Angeles,CA,11922389,34.1141,-118.4068
2,Chicago,IL,8497759,41.8375,-87.6866
3,Miami,FL,6080145,25.7840,-80.2101
4,Houston,TX,5970127,29.7860,-95.3885
...,...,...,...,...,...
75,Grand Rapids,MI,609023,42.9619,-85.6562
76,Columbia,SC,604065,34.0378,-80.9036
77,Albany,NY,590694,42.6664,-73.7987
78,Bakersfield,CA,571992,35.3529,-119.0359


### Find nearest city by using k nearest neighbor (k = 80)

In [4]:
major_city_df['coordinates'] = list(zip(major_city_df['lat'], major_city_df['lng']))
cleaned_housing_data['house_coordinates'] = list(zip(cleaned_housing_data['Latitude'], cleaned_housing_data['Longitude']))

In [5]:
k_neighbors = 80

In [6]:
knn = NearestNeighbors(n_neighbors = k_neighbors, metric = 'euclidean')
knn.fit(major_city_df['coordinates'].values.tolist())

NearestNeighbors(metric='euclidean', n_neighbors=80)

In [8]:
distances, indices = knn.kneighbors(cleaned_housing_data['house_coordinates'].values.tolist())

In [9]:
cleaned_housing_data['closest_city'] = [major_city_df['city'].iloc[idx] for idx in indices[:, 0]]
cleaned_housing_data

,State,City,Street,Zipcode,Bedroom,Bathroom,Area,PPSq,LotArea,MarketEstimate,...,density,source,military,incorporated,timezone,ranking,zips,id,house_coordinates,closest_city
0,AL,Saraland,Scott Dr,36571.0,4.0,2.0,1614.0,148.636927,0.3805,240600.0,...,193.5,shape,False,True,America/Chicago,3,36571,1840015886,"(30.819534, -88.09596)",New Orleans
1,AL,Saraland,Deer Run Dr,36571.0,3.0,2.0,1828.0,108.315098,0.4658,199700.0,...,193.5,shape,False,True,America/Chicago,3,36571,1840015886,"(30.841074, -88.08948)",New Orleans
2,AL,Chelsea,Mallet Way,35043.0,3.0,3.0,2224.0,150.629496,0.2600,336200.0,...,181.7,shape,False,True,America/Chicago,3,35051 35043,1840014823,"(33.357986, -86.6087)",Birmingham
3,AL,Chelsea,Highway 440,35043.0,3.0,2.0,1425.0,195.789474,1.8200,280200.0,...,181.7,shape,False,True,America/Chicago,3,35051 35043,1840014823,"(33.354427, -86.59509)",Birmingham
4,AL,Montgomery,Brampton Ln,36117.0,3.0,2.0,1564.0,96.547315,0.2000,150500.0,...,482.6,shape,False,True,America/Chicago,2,36112 36113 36110 36111 36116 36114 36115 3610...,1840008353,"(32.372746, -86.165115)",Birmingham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12599,WY,Big Piney,Taylor Ave,83113.0,4.0,2.0,1820.0,112.637363,0.1600,189800.0,...,342.1,shape,False,True,America/Denver,3,83113,1840020061,"(42.552986, -110.113075)",Ogden
12600,WY,Daniel,Booth Dr,83115.0,1.0,1.0,1008.0,337.202381,10.0600,307500.0,...,12.0,shape,False,False,America/Denver,3,83115,1840018659,"(42.975636, -110.32426)",Ogden
12601,WY,Shoshoni,Pine St,82649.0,6.0,3.0,3294.0,74.377656,0.3200,236800.0,...,69.0,shape,False,True,America/Denver,3,82649,1840022355,"(43.23948, -108.11593)",Ogden
12602,WY,Mills,N 4th Ave,82604.0,4.0,2.0,2120.0,90.094340,0.3100,184600.0,...,319.8,shape,False,True,America/Denver,3,82644 82604 82601,1840022359,"(42.851006, -106.39181)",Denver


In [10]:
major_city_df

,city,state_id,population,lat,lng,coordinates
0,New York,NY,18908608,40.6943,-73.9249,"(40.6943, -73.9249)"
1,Los Angeles,CA,11922389,34.1141,-118.4068,"(34.1141, -118.4068)"
2,Chicago,IL,8497759,41.8375,-87.6866,"(41.8375, -87.6866)"
3,Miami,FL,6080145,25.7840,-80.2101,"(25.784, -80.2101)"
4,Houston,TX,5970127,29.7860,-95.3885,"(29.786, -95.3885)"
...,...,...,...,...,...,...
75,Grand Rapids,MI,609023,42.9619,-85.6562,"(42.9619, -85.6562)"
76,Columbia,SC,604065,34.0378,-80.9036,"(34.0378, -80.9036)"
77,Albany,NY,590694,42.6664,-73.7987,"(42.6664, -73.7987)"
78,Bakersfield,CA,571992,35.3529,-119.0359,"(35.3529, -119.0359)"


In [15]:
major_city_df = major_city_df.set_index('city')
major_city_df

,state_id,population,lat,lng,coordinates
city,,,,,
New York,NY,18908608,40.6943,-73.9249,"(40.6943, -73.9249)"
Los Angeles,CA,11922389,34.1141,-118.4068,"(34.1141, -118.4068)"
Chicago,IL,8497759,41.8375,-87.6866,"(41.8375, -87.6866)"
Miami,FL,6080145,25.7840,-80.2101,"(25.784, -80.2101)"
Houston,TX,5970127,29.7860,-95.3885,"(29.786, -95.3885)"
...,...,...,...,...,...
Grand Rapids,MI,609023,42.9619,-85.6562,"(42.9619, -85.6562)"
Columbia,SC,604065,34.0378,-80.9036,"(34.0378, -80.9036)"
Albany,NY,590694,42.6664,-73.7987,"(42.6664, -73.7987)"


In [18]:
major_city_df.loc['New York'].get('coordinates')

(40.6943, -73.9249)

In [21]:
cleaned_housing_data

,State,City,Street,Zipcode,Bedroom,Bathroom,Area,PPSq,LotArea,MarketEstimate,...,density,source,military,incorporated,timezone,ranking,zips,id,house_coordinates,closest_city
0,AL,Saraland,Scott Dr,36571.0,4.0,2.0,1614.0,148.636927,0.3805,240600.0,...,193.5,shape,False,True,America/Chicago,3,36571,1840015886,"(30.819534, -88.09596)",New Orleans
1,AL,Saraland,Deer Run Dr,36571.0,3.0,2.0,1828.0,108.315098,0.4658,199700.0,...,193.5,shape,False,True,America/Chicago,3,36571,1840015886,"(30.841074, -88.08948)",New Orleans
2,AL,Chelsea,Mallet Way,35043.0,3.0,3.0,2224.0,150.629496,0.2600,336200.0,...,181.7,shape,False,True,America/Chicago,3,35051 35043,1840014823,"(33.357986, -86.6087)",Birmingham
3,AL,Chelsea,Highway 440,35043.0,3.0,2.0,1425.0,195.789474,1.8200,280200.0,...,181.7,shape,False,True,America/Chicago,3,35051 35043,1840014823,"(33.354427, -86.59509)",Birmingham
4,AL,Montgomery,Brampton Ln,36117.0,3.0,2.0,1564.0,96.547315,0.2000,150500.0,...,482.6,shape,False,True,America/Chicago,2,36112 36113 36110 36111 36116 36114 36115 3610...,1840008353,"(32.372746, -86.165115)",Birmingham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12599,WY,Big Piney,Taylor Ave,83113.0,4.0,2.0,1820.0,112.637363,0.1600,189800.0,...,342.1,shape,False,True,America/Denver,3,83113,1840020061,"(42.552986, -110.113075)",Ogden
12600,WY,Daniel,Booth Dr,83115.0,1.0,1.0,1008.0,337.202381,10.0600,307500.0,...,12.0,shape,False,False,America/Denver,3,83115,1840018659,"(42.975636, -110.32426)",Ogden
12601,WY,Shoshoni,Pine St,82649.0,6.0,3.0,3294.0,74.377656,0.3200,236800.0,...,69.0,shape,False,True,America/Denver,3,82649,1840022355,"(43.23948, -108.11593)",Ogden
12602,WY,Mills,N 4th Ave,82604.0,4.0,2.0,2120.0,90.094340,0.3100,184600.0,...,319.8,shape,False,True,America/Denver,3,82644 82604 82601,1840022359,"(42.851006, -106.39181)",Denver


In [30]:
city_coordinates_list = []
for city in cleaned_housing_data['closest_city']:
    city_coordinates_list.append(major_city_df.loc[city].get('coordinates'))
    #city_coordinates_list = np.append(city_coordinates_list, major_city_df.loc[city].get('coordinates'))
city_coordinates_list

[(30.0687, -89.9288),
 (30.0687, -89.9288),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (33.5279, -86.7971),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687, -89.9288),
 (30.0687,

In [31]:
cleaned_housing_data['city_coordinates'] = city_coordinates_list
cleaned_housing_data

,State,City,Street,Zipcode,Bedroom,Bathroom,Area,PPSq,LotArea,MarketEstimate,...,source,military,incorporated,timezone,ranking,zips,id,house_coordinates,closest_city,city_coordinates
0,AL,Saraland,Scott Dr,36571.0,4.0,2.0,1614.0,148.636927,0.3805,240600.0,...,shape,False,True,America/Chicago,3,36571,1840015886,"(30.819534, -88.09596)",New Orleans,"(30.0687, -89.9288)"
1,AL,Saraland,Deer Run Dr,36571.0,3.0,2.0,1828.0,108.315098,0.4658,199700.0,...,shape,False,True,America/Chicago,3,36571,1840015886,"(30.841074, -88.08948)",New Orleans,"(30.0687, -89.9288)"
2,AL,Chelsea,Mallet Way,35043.0,3.0,3.0,2224.0,150.629496,0.2600,336200.0,...,shape,False,True,America/Chicago,3,35051 35043,1840014823,"(33.357986, -86.6087)",Birmingham,"(33.5279, -86.7971)"
3,AL,Chelsea,Highway 440,35043.0,3.0,2.0,1425.0,195.789474,1.8200,280200.0,...,shape,False,True,America/Chicago,3,35051 35043,1840014823,"(33.354427, -86.59509)",Birmingham,"(33.5279, -86.7971)"
4,AL,Montgomery,Brampton Ln,36117.0,3.0,2.0,1564.0,96.547315,0.2000,150500.0,...,shape,False,True,America/Chicago,2,36112 36113 36110 36111 36116 36114 36115 3610...,1840008353,"(32.372746, -86.165115)",Birmingham,"(33.5279, -86.7971)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12599,WY,Big Piney,Taylor Ave,83113.0,4.0,2.0,1820.0,112.637363,0.1600,189800.0,...,shape,False,True,America/Denver,3,83113,1840020061,"(42.552986, -110.113075)",Ogden,"(41.2279, -111.9682)"
12600,WY,Daniel,Booth Dr,83115.0,1.0,1.0,1008.0,337.202381,10.0600,307500.0,...,shape,False,False,America/Denver,3,83115,1840018659,"(42.975636, -110.32426)",Ogden,"(41.2279, -111.9682)"
12601,WY,Shoshoni,Pine St,82649.0,6.0,3.0,3294.0,74.377656,0.3200,236800.0,...,shape,False,True,America/Denver,3,82649,1840022355,"(43.23948, -108.11593)",Ogden,"(41.2279, -111.9682)"
12602,WY,Mills,N 4th Ave,82604.0,4.0,2.0,2120.0,90.094340,0.3100,184600.0,...,shape,False,True,America/Denver,3,82644 82604 82601,1840022359,"(42.851006, -106.39181)",Denver,"(39.762, -104.8758)"


### Calculate distance between city and house

In [42]:
cleaned_housing_data.columns

Index(['State', 'City', 'Street', 'Zipcode', 'Bedroom', 'Bathroom', 'Area',
       'PPSq', 'LotArea', 'MarketEstimate', 'RentEstimate', 'Latitude',
       'Longitude', 'ListedPrice', 'population', 'density', 'source',
       'military', 'incorporated', 'timezone', 'ranking', 'zips', 'id',
       'house_coordinates', 'closest_city', 'city_coordinates'],
      dtype='object')

In [45]:
house_lat = cleaned_housing_data['Latitude']
house_lng = cleaned_housing_data['Longitude']


In [47]:
cleaned_housing_data['city_coordinates'][0][0]

30.0687

In [53]:
city_lat = []
city_lng = []
for i in cleaned_housing_data['city_coordinates']:
    city_lat.append(i[0])
    city_lng.append(i[1])

[-89.9288,
 -89.9288,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -89.9288,
 -89.9288,
 -89.9288,
 -89.9288,
 -84.422,
 -86.7842,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -86.7971,
 -89.9288,
 -89.9288,


In [58]:
distance_list = []
for i in range(cleaned_housing_data.shape[0]):
    distance_list.append(geodesic((house_lat[i], house_lng[i]), (city_lat[i], city_lng[i])).miles)
distance_list

[121.00141758365082,
 121.98004519544847,
 15.987941468206486,
 16.70813684463997,
 87.66442788520114,
 83.80609123554292,
 91.92084724380585,
 87.00798631927468,
 83.0076258813861,
 84.5299591364844,
 87.77122503241891,
 88.31317104956902,
 86.0513348953353,
 85.03873111207365,
 87.69356047093764,
 90.91625590195315,
 85.05236473852153,
 60.430385653427656,
 56.00950677194911,
 61.3370083909122,
 109.30541163980648,
 118.06210680715375,
 118.22881647859894,
 115.39768224073546,
 113.30470878269207,
 118.3337964515316,
 118.55540855281149,
 117.29944040814156,
 112.84194068351783,
 106.45636329102825,
 110.08306033538388,
 111.59605612158842,
 109.64726573027075,
 112.89191471679972,
 116.66068482583866,
 115.2815806840311,
 111.65882395857622,
 113.11496136918541,
 119.01197418061675,
 107.75762158614403,
 103.73740417942453,
 117.8403332212,
 109.50168794719515,
 106.24262491034189,
 107.82473421218796,
 107.38763390614741,
 103.09743007130164,
 119.35818831428888,
 112.8505285546272

In [59]:
cleaned_housing_data['distance_to_maj_city_miles'] = distance_list
cleaned_housing_data

,State,City,Street,Zipcode,Bedroom,Bathroom,Area,PPSq,LotArea,MarketEstimate,...,military,incorporated,timezone,ranking,zips,id,house_coordinates,closest_city,city_coordinates,distance_to_maj_city_miles
0,AL,Saraland,Scott Dr,36571.0,4.0,2.0,1614.0,148.636927,0.3805,240600.0,...,False,True,America/Chicago,3,36571,1840015886,"(30.819534, -88.09596)",New Orleans,"(30.0687, -89.9288)",121.001418
1,AL,Saraland,Deer Run Dr,36571.0,3.0,2.0,1828.0,108.315098,0.4658,199700.0,...,False,True,America/Chicago,3,36571,1840015886,"(30.841074, -88.08948)",New Orleans,"(30.0687, -89.9288)",121.980045
2,AL,Chelsea,Mallet Way,35043.0,3.0,3.0,2224.0,150.629496,0.2600,336200.0,...,False,True,America/Chicago,3,35051 35043,1840014823,"(33.357986, -86.6087)",Birmingham,"(33.5279, -86.7971)",15.987941
3,AL,Chelsea,Highway 440,35043.0,3.0,2.0,1425.0,195.789474,1.8200,280200.0,...,False,True,America/Chicago,3,35051 35043,1840014823,"(33.354427, -86.59509)",Birmingham,"(33.5279, -86.7971)",16.708137
4,AL,Montgomery,Brampton Ln,36117.0,3.0,2.0,1564.0,96.547315,0.2000,150500.0,...,False,True,America/Chicago,2,36112 36113 36110 36111 36116 36114 36115 3610...,1840008353,"(32.372746, -86.165115)",Birmingham,"(33.5279, -86.7971)",87.664428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12599,WY,Big Piney,Taylor Ave,83113.0,4.0,2.0,1820.0,112.637363,0.1600,189800.0,...,False,True,America/Denver,3,83113,1840020061,"(42.552986, -110.113075)",Ogden,"(41.2279, -111.9682)",132.340884
12600,WY,Daniel,Booth Dr,83115.0,1.0,1.0,1008.0,337.202381,10.0600,307500.0,...,False,False,America/Denver,3,83115,1840018659,"(42.975636, -110.32426)",Ogden,"(41.2279, -111.9682)",147.269952
12601,WY,Shoshoni,Pine St,82649.0,6.0,3.0,3294.0,74.377656,0.3200,236800.0,...,False,True,America/Denver,3,82649,1840022355,"(43.23948, -108.11593)",Ogden,"(41.2279, -111.9682)",241.450936
12602,WY,Mills,N 4th Ave,82604.0,4.0,2.0,2120.0,90.094340,0.3100,184600.0,...,False,True,America/Denver,3,82644 82604 82601,1840022359,"(42.851006, -106.39181)",Denver,"(39.762, -104.8758)",227.287741


In [61]:
cleaned_housing_data

,State,City,Street,Zipcode,Bedroom,Bathroom,Area,PPSq,LotArea,MarketEstimate,...,military,incorporated,timezone,ranking,zips,id,house_coordinates,closest_city,city_coordinates,distance_to_maj_city_miles
0,AL,Saraland,Scott Dr,36571.0,4.0,2.0,1614.0,148.636927,0.3805,240600.0,...,False,True,America/Chicago,3,36571,1840015886,"(30.819534, -88.09596)",New Orleans,"(30.0687, -89.9288)",121.001418
1,AL,Saraland,Deer Run Dr,36571.0,3.0,2.0,1828.0,108.315098,0.4658,199700.0,...,False,True,America/Chicago,3,36571,1840015886,"(30.841074, -88.08948)",New Orleans,"(30.0687, -89.9288)",121.980045
2,AL,Chelsea,Mallet Way,35043.0,3.0,3.0,2224.0,150.629496,0.2600,336200.0,...,False,True,America/Chicago,3,35051 35043,1840014823,"(33.357986, -86.6087)",Birmingham,"(33.5279, -86.7971)",15.987941
3,AL,Chelsea,Highway 440,35043.0,3.0,2.0,1425.0,195.789474,1.8200,280200.0,...,False,True,America/Chicago,3,35051 35043,1840014823,"(33.354427, -86.59509)",Birmingham,"(33.5279, -86.7971)",16.708137
4,AL,Montgomery,Brampton Ln,36117.0,3.0,2.0,1564.0,96.547315,0.2000,150500.0,...,False,True,America/Chicago,2,36112 36113 36110 36111 36116 36114 36115 3610...,1840008353,"(32.372746, -86.165115)",Birmingham,"(33.5279, -86.7971)",87.664428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12599,WY,Big Piney,Taylor Ave,83113.0,4.0,2.0,1820.0,112.637363,0.1600,189800.0,...,False,True,America/Denver,3,83113,1840020061,"(42.552986, -110.113075)",Ogden,"(41.2279, -111.9682)",132.340884
12600,WY,Daniel,Booth Dr,83115.0,1.0,1.0,1008.0,337.202381,10.0600,307500.0,...,False,False,America/Denver,3,83115,1840018659,"(42.975636, -110.32426)",Ogden,"(41.2279, -111.9682)",147.269952
12601,WY,Shoshoni,Pine St,82649.0,6.0,3.0,3294.0,74.377656,0.3200,236800.0,...,False,True,America/Denver,3,82649,1840022355,"(43.23948, -108.11593)",Ogden,"(41.2279, -111.9682)",241.450936
12602,WY,Mills,N 4th Ave,82604.0,4.0,2.0,2120.0,90.094340,0.3100,184600.0,...,False,True,America/Denver,3,82644 82604 82601,1840022359,"(42.851006, -106.39181)",Denver,"(39.762, -104.8758)",227.287741


In [62]:
cleaned_housing_data.to_csv('cleaned_housing_data.csv', index = False)

In [63]:
cleaned_housing_data.sort_values(by = 'ranking')

,State,City,Street,Zipcode,Bedroom,Bathroom,Area,PPSq,LotArea,MarketEstimate,...,military,incorporated,timezone,ranking,zips,id,house_coordinates,closest_city,city_coordinates,distance_to_maj_city_miles
11202,VA,Virginia Beach,Chayote Ct #20,23462.0,3.0,3.0,1994.0,138.415246,0.030000,279400.0,...,False,True,America/New_York,1,23459 23451 23453 23452 23455 23457 23456 2345...,1840003871,"(36.85527, -76.15303)",Virginia Beach,"(36.7335, -76.0435)",10.363527
6069,MO,Kansas City,Bridlespur Dr,64114.0,4.0,4.0,3706.0,76.902321,0.265794,286900.0,...,False,True,America/Chicago,1,64161 64163 64164 64165 64167 64053 64118 6411...,1840008535,"(38.933414, -94.59213)",Kansas City,"(39.1238, -94.5541)",13.291687
6068,MO,Kansas City,NW Briarcliff Pkwy,64116.0,4.0,6.0,6700.0,119.253731,0.539991,764600.0,...,False,True,America/Chicago,1,64161 64163 64164 64165 64167 64053 64118 6411...,1840008535,"(39.169983, -94.5869)",Kansas City,"(39.1238, -94.5541)",3.640598
1157,CA,San Diego,Pantera Rd,92130.0,3.0,2.0,1620.0,1358.024691,0.093388,2261800.0,...,False,True,America/Los_Angeles,1,92182 92109 92108 92107 92106 92103 92101 9211...,1840021990,"(32.959915, -117.24034)",San Diego,"(32.8313, -117.1222)",11.212866
1156,CA,San Diego,Morning Sage Way,92130.0,5.0,5.0,4177.0,948.048839,0.186042,3983300.0,...,False,True,America/Los_Angeles,1,92182 92109 92108 92107 92106 92103 92101 9211...,1840021990,"(32.971455, -117.197075)",San Diego,"(32.8313, -117.1222)",10.593853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8053,NC,Nags Head,W Cobbs Way,27959.0,3.0,3.0,1570.0,381.528662,0.275482,602300.0,...,False,True,America/New_York,4,27959,1840016492,"(35.94886, -75.626274)",Virginia Beach,"(36.7335, -76.0435)",58.895032
201,AL,Dadeville,Gibson Rd,36853.0,2.0,2.0,1082.0,170.979667,4.400000,173500.0,...,False,True,America/Chicago,4,36853,1840001397,"(32.83792, -85.79874)",Birmingham,"(33.5279, -86.7971)",74.886291
3190,IN,Marion,E 61st St,46953.0,3.0,2.0,1474.0,101.696065,0.458000,149800.0,...,False,True,America/Indiana/Indianapolis,4,46176,1840001580,"(40.49694, -85.6344)",Indianapolis,"(39.7771, -86.1458)",56.568846
7581,NM,Edgewood,Dinkle Rd,87015.0,3.0,2.0,1353.0,236.511456,13.910000,312000.0,...,False,True,America/Denver,4,87047 87015,1840074899,"(35.08403, -106.11915)",Albuquerque,"(35.1054, -106.6465)",29.915066
